En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [37]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

df = pd.read_json('/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4.json', lines=True)

df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month.apply(lambda x: f'{x:02}')
df['day'] = df['date'].dt.day.apply(lambda x: f'{x:02}')

chunk_size = 5000

output_path = '/Users/juanignaciomagarinoscastro/Downloads/tweets_by_date'

In [32]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

for (year, month, day), group in df.groupby(['year', 'month', 'day']):
    partition_path = os.path.join(output_path, f'year={year}', f'month={month}', f'day={day}')
    
    if not os.path.exists(partition_path):
        os.makedirs(partition_path)
    
    num_chunks = (len(group) // chunk_size) + 1
    for i in range(num_chunks):
        start_row = i * chunk_size
        end_row = min(start_row + chunk_size, len(group))
        chunk = group.iloc[start_row:end_row]
        
        chunk.to_parquet(
            os.path.join(partition_path, f'farmers-protest-tweets-{year}{month}{day}_part_{i}.parquet'),
            engine='pyarrow',
            index=False
        )